In [24]:
import pandas as pd
import numpy as np
import pickle
import time
import random

import itertools

In [2]:
%%time
graph_dict = pd.read_feather('data/graph_128.ftr', use_threads=True).set_index('source_item_id').target_item_ids
print(len(graph_dict))
graph_dict[9036]

38488233
CPU times: user 16.2 s, sys: 5.43 s, total: 21.7 s
Wall time: 21.2 s


array([ 1132636,       60,   367211,        5, 12750167, 16086586,
       16098713, 37251206,   131566,   169470,    38104,    31519,
         205375,   632404,  1366018,  1517792,  5442484,   689775,
          93728,  1332315,   678263,  3332215,  1905051,  4920135,
         111734,  1326886,  1906857,  2267418,  3739104, 17552952,
       41799198,       90,     1085,     1781,    12152,   127885,
          43035,   101333,    49258,    13298,  1280677,  1700481,
           3617,   613201,     1860,     9301,       30,      397,
            150,      188,      652,    28513,     9067,   131964,
           9056,    83364,    93996,  5460604])

In [3]:
%%time
sentences = pickle.load(open("data/dataset.p", "rb"))
print(len(sentences))
sentences[0]

1000
CPU times: user 13 ms, sys: 128 µs, total: 13.1 ms
Wall time: 12.3 ms


([array([ 1194829,  7269253,  6841945,  5177761,  7456161, 26377050,
         17554942,  7456157,  7456158,  7456162, 55046998, 26668131,
         17551826, 26317225, 26526504, 26568131, 26640846, 17543594]),
  array([     362,   713750,   769744,   216108,    48837,    35615,
            23311,   713493,    34493,   107802,   795578,     7603,
           152323,   134949,  1242743,   273199,  4440864,  1860150,
          6589727,   272220,   687214,     3913,  3075752,    19095,
           222032,  1756146,   191494,   184768,   233008,   738019,
           997459,  6453233,  1083063,  1861637,   187266,  1232349,
            51788,  7947559,  2657116,     8853,  1935842,    39911,
          1416835,   622964,   110022,   699201,   109331,  8565258,
          7878662,      200,   989445,  3063504,  1770746,   617167,
            11694,  3994977,  1965838,  3564702,   590944,    53113,
           233132,  3564704,  5247677,   715731,  7043346,    48423,
           719915,  7952640,  12

In [4]:
id_counts_df = pd.read_csv("data/id_counts.csv").set_index('item_id')
display(id_counts_df)
item_counts_dict = id_counts_df.counts.to_dict()
print(item_counts_dict[6199])

,page_id,title,views,counts
item_id,,,,
6199,12,Anarchism,31335,3540
38404,25,Autism,49693,2114
101038,39,Albedo,14573,2825
9659,290,A,25859,175
173,303,Alabama,52765,11125
...,...,...,...,...
76894635,62470350,Daming Zhu,16,0
76894633,62470423,Tony Dews,7,2
76896959,62470432,Samsung PL20,9,0


3540


In [5]:
def sort_ids(candidate_ids, n):
    if len(candidate_ids) == 0:
        return candidate_ids
    tie_break_list = [item_counts_dict.get(i, 0) for i in candidate_ids]
    if n > 1:
        return candidate_ids[np.argsort(tie_break_list)[::-1]][:n]
    else:
        return candidate_ids[np.argmax(tie_break_list)]
    
def get_children(item_id, max_children):
    return graph_dict.get(item_id, np.array([], dtype='int32'))[:max_children]

def get_next_level(level_graph, max_width, max_children):
    next_level = pd.unique(np.concatenate([get_children(i, max_children) for i in level_graph[-1]]))
    next_level = np.setdiff1d(next_level, np.concatenate(level_graph))

    if len(next_level) > max_width:
        return sort_ids(next_level, n=max_width)

    return next_level
    
def traverse_graph(root_id, match_id, max_depth, max_width, max_children, verbose=False):
    match = root_id == match_id
    depth = 0
    level = [[root_id]]
    
    st = time.time()
    while match == False and depth < max_depth:
        depth += 1
        if verbose: print(f'Depth: {depth}\tTime: {time.time()-st}')
        if len(level[-1]) == 0:
            break
            
        # Get the next level of the graph
        level.append(get_next_level(level, max_width, max_children))
        
        if match_id is not None and match_id in level[-1]:
            match = True
            break
        
    if verbose: print(f'Traversed {depth} levels in {time.time()-st}')
    if verbose: [print(f'Level: {i}\tLength: {len(j)}') for i, j in enumerate(level)]
        
    if match == True:
        if verbose: print(f'Match Found! {match_id, level[-1]}')
        return depth
    return max_depth+5

# 9036 Nikola Tesla
# 478214 Tesla Car Company
traverse_graph(root_id=9036, match_id=478214, max_depth=8, max_width=7200, max_children=24, verbose=True)

Depth: 1	Time: 1.430511474609375e-06
Depth: 2	Time: 0.0004734992980957031
Depth: 3	Time: 0.001898050308227539
Depth: 4	Time: 0.009551286697387695
Depth: 5	Time: 0.048937082290649414
Depth: 6	Time: 0.2523159980773926
Depth: 7	Time: 0.4940323829650879
Traversed 7 levels in 0.7341458797454834
Level: 0	Length: 1
Level: 1	Length: 24
Level: 2	Length: 200
Level: 3	Length: 1339
Level: 4	Length: 6398
Level: 5	Length: 7200
Level: 6	Length: 7200
Level: 7	Length: 7200
Match Found! (478214, array([ 637413,  128309,  193592, ...,  152418, 2414422,  444990]))


7

In [6]:
%timeit traverse_graph(root_id=9036, match_id=-1, max_depth=8, max_width=7200, max_children=32)

1.03 s ± 2.19 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [7]:
%%time
def reduce_neighbors(sample, max_neighbors, max_neighbor_ids):
    temp = [i for i in sample if len(i)>0]
    if len(temp) <= max_neighbors+1:
        temp = [i for i in temp if i[0] is not None]
    else:
        for i, j in enumerate(temp):
            if j[0] is None:
                if i == 0 or i == len(temp):
                    temp = [i for i in temp if i[0] is not None][:max_neighbors]
                else:
                    left = temp[:i]
                    right = temp[i+1:]
                    if len(left) <= max_neighbors/2:
                        temp = left + right[:max_neighbors-len(left)]
                    elif len(right) <= max_neighbors/2:
                        temp = left[-(max_neighbors-len(right)):] + right
                    else:
                        left = left[-max_neighbors//2:]
                        temp = left + right[:max_neighbors-len(left)]
                break
    temp = [sort_ids(i, max_neighbor_ids) for i in temp]
    return temp

sample = sentences[1][0]
display(sample)
reduced_sample = reduce_neighbors(sample, 2, 8)
reduced_sample

[array([      61,    11268,    48525,    11208,   204627,   223240,
         7889769,   171221,  2305815,  2367175,   466835,  1141049,
          383689, 19985418,   153312,   676576,  1446108,     3570,
          517545,   386613,  5608924,  1365883,  7954909,  1043283,
         7951257,   645710,  7956433,  4711838,  3336946,  6824810,
          299950,  1898341,  3566495, 60769874,  7623083,  6455666,
         1347319,  4573266,   824948,  7951169,  4404065,  7948302,
         5077180,   685093,   137245,  3537231,  1810883,  8010306,
           17471,  4644611, 20715627,  7955261,  1319687,  4887040,
        13724508,  7759592, 33195102,   403782,  1188049,  6942325,
         7949233,  7971599,  5185405,  7579915,  7918583,  7698307,
         7009139,  7971591,  7958265,  7127232, 28222286, 48769328,
          598230, 17015313,  5256101,  5911750,  7835282,   126110,
        28373879,  7233643, 55613805, 24965378,  3835186,  7893153,
        16966843,  3965696,  5911806, 20488868, 

CPU times: user 3.47 ms, sys: 0 ns, total: 3.47 ms
Wall time: 2.88 ms


[array([    361,  163727, 1361864,  324867,   37643,    2141,  754839,
        3707858]),
 array([    91,  23090,  18519, 581044,  28260, 180057,  49142,  81153])]

In [ ]:
def disambiguate(sample, max_neighbors, max_neighbor_ids, max_root_ids, \
                 max_depth, max_width, max_children, verbose=False, baseline=False):
    
    st = time.time()
    
    true_id = sample[2][2]
    candidate_ids = sample[0][0][:max_root_ids]
    neighbor_ids = [i for i in sample[0][1:] if len(i)>0]
        
    if len(candidate_ids) <= 1 or len(neighbor_ids) <= 1 or baseline:
        return sort_ids(candidate_ids, 1) == true_id
    
    print(f'Disambiguating: {true_id}\t{time.time()-st}')

    neighbor_ids = reduce_neighbors(neighbor_ids, max_neighbors, max_neighbor_ids)
    
    comb_ids = [candidate_ids]+neighbor_ids
    combination_list = list(itertools.product(*comb_ids))
    #print(combination_list)
    if verbose: print(len(combination_list), len(list(itertools.combinations(combination_list[0], 2))))
    
    #return
    shortest_dist = np.inf
    pair_dict = {}
    for idx, combination in enumerate(combination_list):
        distance = 0
        pairs = list(itertools.combinations(combination, 2))
        if verbose: print(f'processing {len(pairs)} pairs at idx {idx}\t{time.time()-st}\tcurrent shortest: {shortest_dist}')
        for pair in pairs:
            if pair[0] not in candidate_ids and pair[1] not in candidate_ids:
                continue
            if pair in pair_dict:
                pass
            else:
                pair_dict[pair] = traverse_graph(pair[0], pair[1], max_depth, max_width, max_children)
            
            distance += pair_dict[pair]
            
        if distance < shortest_dist:
            shortest_dist = distance
            candidate_id = combination[0]
            if verbose: print(f'new shortest: {shortest_dist}, candidate_id: {candidate_id}')

        #if shortest_dist <= 2:
            #print(f'Early Break {idx}: {candidate_id} {true_id}')
            #break
    
    if verbose: print(candidate_ids, candidate_id, true_id, time.time()-st)
    return candidate_id == true_id

disambiguate(sentences[3], max_neighbors=4, max_neighbor_ids=8, max_root_ids=8, \
             max_depth=10, max_width=10000, max_children=32, verbose=True)

In [9]:
baseline = [disambiguate(i, max_neighbors=2, max_neighbor_ids=8, max_root_ids=8, 
                         max_depth=8, max_width=7200, max_children=32, baseline=True) for i in sentences]
sum(baseline)/len(baseline)
baseline

Disambiguating: 7456158	5.0067901611328125e-06
Disambiguating: 61	3.337860107421875e-06
Disambiguating: 69834	3.0994415283203125e-06
Disambiguating: 639669	3.5762786865234375e-06
Disambiguating: 796777	2.6226043701171875e-06
Disambiguating: 34740	2.6226043701171875e-06
Disambiguating: 4638682	2.6226043701171875e-06
Disambiguating: 50783	2.384185791015625e-06
Disambiguating: 18677983	3.0994415283203125e-06
Disambiguating: 3719	1.9073486328125e-06
Disambiguating: 9778	2.1457672119140625e-06
Disambiguating: 1326230	2.384185791015625e-06
Disambiguating: 5241252	2.6226043701171875e-06
Disambiguating: 5322628	2.1457672119140625e-06
Disambiguating: 727	1.6689300537109375e-06
Disambiguating: 2454881	1.430511474609375e-06
Disambiguating: 3479072	2.86102294921875e-06
Disambiguating: 600982	1.6689300537109375e-06
Disambiguating: 182531	2.384185791015625e-06
Disambiguating: 204181	2.1457672119140625e-06
Disambiguating: 4421	1.9073486328125e-06
Disambiguating: 269528	2.6226043701171875e-06
Disambig

[False,
 True,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 Tru

In [10]:
import multiprocessing as mp
from functools import partial 
n_cores = mp.cpu_count()
n_cores

8

In [21]:
def get_samples(dataset, n, seed=None):
    random.seed(seed)
    return random.sample(dataset, n)

In [ ]:
%%time

disambiguate_part = partial(disambiguate, max_neighbors=8, max_neighbor_ids=8, max_root_ids=8, 
                            max_depth=3, max_width=100000, max_children=)

samples = get_samples(sentences, 100, 5)
baseline = [disambiguate(i, max_neighbors=4, max_neighbor_ids=8, max_root_ids=8, 
                         max_depth=8, max_width=7200, max_children=32, baseline=True) for i in samples]
print(f'baseline results: {sum(baseline)/len(baseline)}')

st = time.time()
p = mp.Pool(n_cores-1)
results = p.map(disambiguate_part, samples)
p.close()
p.join()

sum(results)/len(results)



baseline results: 0.72
Disambiguating: 5369140	1.4066696166992188e-05
Disambiguating: 135475	3.123283386230469e-05
Disambiguating: 1369951	6.67572021484375e-06
Disambiguating: 1952	7.62939453125e-06
Disambiguating: 1345971	8.821487426757812e-06
Disambiguating: 664	5.9604644775390625e-06
Disambiguating: 11412	9.059906005859375e-06
[ 1345971 20648663] 1345971 1345971 0.026149749755859375
Disambiguating: 49297	5.0067901611328125e-06
[  11412   11452  284602 3497167  169973  134465 2579784  265628] 11412 11412 0.16353201866149902
Disambiguating: 771415	8.821487426757812e-06
[ 1369951 30608234] 1369951 1369951 0.2155287265777588
Disambiguating: 838366	4.5299530029296875e-06
[ 654034  771415 1335080  135715] 1335080 771415 0.18301749229431152
Disambiguating: 390018	6.198883056640625e-06
[  714339   838366  4203777   705963  6158580 52334250 26216346] 838366 838366 0.34392666816711426
Disambiguating: 10387	4.0531158447265625e-06
[ 10387 574567] 10387 10387 0.013842105865478516
Disambiguating:

In [44]:
sum(results)/len(results)

0.7

In [16]:
new_result = [i[1] if type(i) is tuple else i for i in results]
new_result
sum(new_result)/len(new_result)

0.725